In [1]:
# ./elasticsearch-7.12.1/bin/elasticsearch -d

In [2]:
!nvidia-smi

Sun Jul 18 11:16:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3A:00.0 Off |                    0 |
| N/A   32C    P0    59W / 300W |     34MiB / 32510MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [3]:
import sys
import os

#!{sys.executable} -m pip install elasticsearch

import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string
import csv                 
import pickle
import json
import json_lines
import pandas as pd
import math

from six import string_types
from typing import Dict
from pathlib import Path
from functools import partial
from collections import OrderedDict
from argparse import ArgumentParser

import tqdm

import haystack
import elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.sparse import ElasticsearchRetriever

import lineflow as lf
from transformers import AlbertForMultipleChoice, AlbertTokenizer, AdamW, AlbertConfig, AlbertModel
import pytorch_lightning as pl
from pytorch_lightning.metrics import functional as FM

import torch
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

import time

from pytorch_lightning import seed_everything
import random


07/18/2021 11:16:31 - INFO - faiss.loader -   Loading faiss with AVX2 support.
07/18/2021 11:16:31 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
07/18/2021 11:16:31 - INFO - faiss.loader -   Loading faiss.
07/18/2021 11:16:31 - INFO - faiss.loader -   Successfully loaded faiss.
07/18/2021 11:16:31 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/pfs/data5/home/st/st_us-051200/st_st171103/env2hay/lib64/python3.6/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


In [4]:
seed = 42

seed_everything(seed, workers=True)

# python RNG
import random
random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

07/18/2021 11:16:32 - INFO - pytorch_lightning.utilities.seed -   Global seed set to 42


In [5]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import Dense, Input, TimeDistributed, Flatten, LSTM, Bidirectional, GRU, ConvLSTM2D 
from tensorflow.keras.layers import Concatenate, Dropout, SpatialDropout1D, SpatialDropout2D
from tensorflow.keras.models import Model

import tensorflow as tf

from datetime import datetime
from tensorflow import keras
import jsonlines
import argparse
from os import listdir
from os.path import isfile, join

import uuid

/opt/bwhpc/common/jupyter/tensorflow/2021-07-05/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
from keras.callbacks import ModelCheckpoint

In [8]:
tf.random.set_seed(42)

In [9]:
!curl localhost:9200

{
  "name" : "node-1",
  "cluster_name" : "ARC-cluster",
  "cluster_uuid" : "OKjJu8d1R7GBO1mjNE0iEw",
  "version" : {
    "number" : "7.12.1",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "3186837139b9c6b6d23c3200870651f10d3343b7",
    "build_date" : "2021-04-20T20:56:39.040728659Z",
    "build_snapshot" : false,
    "lucene_version" : "8.8.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [10]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

07/18/2021 11:16:47 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.007s]
07/18/2021 11:16:47 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.001s]
07/18/2021 11:16:47 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.001s]
07/18/2021 11:16:47 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.004s]
07/18/2021 11:16:47 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.001s]


In [11]:
document_store.get_document_count()

07/18/2021 11:16:47 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.002s]


19670197

In [12]:
retriever = ElasticsearchRetriever(document_store=document_store)

In [13]:
MAX_LEN = 512
NUM_LABELS = 4
label_map = {"A": 0, "B": 1, "C": 2, "D": 3}
BATCH_SIZE = 8

In [14]:
def raw_samples_to_dataset(samples, daset):
    datas = []
    i = 0
    for sample in samples:
        i+=1
        if i%250 == 0:
            print(i)

        _uniques = []
        _rscore = []
        _article = []
        _options = []
        toomany = 0
        
        _id = sample["id"]          
        
        _answer = sample["answerKey"]        
        if _answer == "5" or _answer == "E":
            toomany = random.randint(1, 4)   
            _answer = str(toomany)
            print(_id)
            print("Too many :")
            print(toomany)  
        if _answer == "1":
            _answer = "A"
        if _answer == "2":
            _answer = "B"
        if _answer == "3":
            _answer = "C"
        if _answer == "4":
            _answer = "D"        
        if _answer not in ["A", "B", "C", "D"]:
            print(_id)
            print("skipped")
            continue
        assert(_answer in ["A", "B", "C", "D"])        
        
        _question = sample["question"]['stem']   
        for idx in range(len(sample['question']['choices'])): 
            answer = sample["question"]['choices'][idx]['label']
            if answer == "1":
                answer = "A"
            if answer == "2":
                answer = "B"
            if answer == "3":
                answer = "C"
            if answer == "4":
                answer = "D"            
            if answer not in ["A", "B", "C", "D"]:
                continue                
            if toomany > 0 and _answer == answer:
                _options.append(sample["question"]['choices'][4]['text'])
                continue          
            _options.append(sample["question"]['choices'][idx]['text'])            

        if len(_options) < 4:
            print(_id)
            print("skipped, less than 4 options")
            continue
        assert(len(_options) == 4)            

        for idx in range(len(_options)): 
            _facts = ""
            _scores = []
            for result in retriever.retrieve(query= _question + " " + _options[idx], top_k=4):
                _facts = _facts + " " + result.text
                _scores.append(result.score)
            _rscore.append(_scores)
            _article.append(_facts)
                
        data = {
                "id": _id,
                "article": _article,
                "options": _options,
                "question": _question,
                "answer": _answer,
                "optionscore": [],
                "final_optionscore": [],
                "rscore": _rscore,
                "chosen": [],
                "chosen_index": []
                }
        datas.append(data)
        daset.append(data)
    return lf.Dataset(datas)   

In [15]:
def preprocess(tokenizer: AlbertTokenizer, x: Dict) -> Dict:

    choices_features = []
    i=0
    option: str
    for option in x["options"]:
        text_a = x["article"][i]
        i+=1
        text_b = x["question"] + " " + option
        
        inputs = tokenizer.encode_plus(
                text_a,
                text_b,
                add_special_tokens=True,
                max_length=MAX_LEN
                )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        attention_mask = [1] * len(input_ids)

        pad_token_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)
        input_ids = input_ids + ([pad_token_id] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_id] * padding_length)

        assert len(input_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(input_ids), MAX_LEN)
        assert len(attention_mask) == MAX_LEN, "Error with input length {} vs {}".format(len(attention_mask), MAX_LEN)
        assert len(token_type_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(token_type_ids), MAX_LEN)

        choices_features.append({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            })

    labels = label_map.get(x["answer"], -1)
    label = torch.tensor(labels).long()

    return {
            "id": x["id"],
            "label": label,
            "input_ids": torch.tensor([cf["input_ids"] for cf in choices_features]),
            "attention_mask": torch.tensor([cf["attention_mask"] for cf in choices_features]),
            "token_type_ids": torch.tensor([cf["token_type_ids"] for cf in choices_features]),
            }

In [16]:
def get_demo_dataloader(tokenizer, datadir: str = './', cachedir: str = "./"):
    datadir = Path(datadir)
    datafile = "test_demo_day_10.jsonl"
    cachedir = Path(cachedir)
    filename = "_demo_"
    cachefile = "cache" + filename + ".cache"
    ds_file = "DS" + filename + ".pkl"
    dl_file = "DL" + filename + ".pkl"
    
    if os.path.isfile(cachedir / cachefile):
        os.remove(cachedir / cachefile)
    if os.path.isfile(ds_file):
        os.remove(ds_file)
    if os.path.isfile(dl_file):
        os.remove(dl_file)
        
    preprocessor = partial(preprocess, tokenizer)

    test_samples = []
    with open(datadir / datafile) as f:
        for item in json_lines.reader(f):
            test_samples.append(item)
    test = raw_samples_to_dataset(test_samples,t_dataset)
    pickle.dump(test, open(ds_file, "wb"))   
    test_dataloader = DataLoader(
            test.map(preprocessor).save(cachedir / cachefile),
            sampler=SequentialSampler(test),
            batch_size=BATCH_SIZE
            )
    
    pickle.dump(test_dataloader, open(dl_file, "wb"))
    return test_dataloader


In [17]:
class TestModel(pl.LightningModule):

    def __init__(self, model, test_dataloader, model2):
        super(TestModel, self).__init__()

        self.model = model
        self.model2 = model2
        self._test_dataloader = test_dataloader

    def test_step(self, batch, batch_idx):
        ids = batch["id"]
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
        )
        outputs_pooler = self.model2(input_ids = torch.reshape(input_ids,(-1,512)),
                           token_type_ids=torch.reshape(token_type_ids,(-1,512)),
                           attention_mask=torch.reshape(attention_mask,(-1,512)))
        
        labels_hat = torch.argmax(outputs.logits, dim=1)

        i = 0
        j = int(len(batch["id"])/torch.cuda.device_count())

        for element in outputs.logits:
            for element2 in t_dataset:
                if j == 0 and element2["id"] == batch["id"][torch.cuda.current_device()]:
                    hidden_pos = i*4
                    element2["final_optionscore"].append(element.cpu().numpy())
                    element2["optionscore"] = outputs_pooler.pooler_output[hidden_pos:hidden_pos+4].cpu().numpy()
                    element2["chosen"].append(labels_hat.cpu().numpy()[i])    
                elif j != 0 and element2["id"] == batch["id"][torch.cuda.current_device()*j+i]:
                    hidden_pos = i*4
                    element2["final_optionscore"].append(element.cpu().numpy())
                    element2["optionscore"] = outputs_pooler.pooler_output[hidden_pos:hidden_pos+4].cpu().numpy()
                    element2["chosen"].append(labels_hat.cpu().numpy()[i])    
            i+=1
        acc = FM.accuracy(labels_hat, labels)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def test_dataloader(self):
        return self._test_dataloader

In [18]:
def Generate_NN_file(src ,nn_file, runs):
    if os.path.isfile(nn_file):
        os.remove(nn_file)
    checkpoint_path = src
    checkpoint = torch.load(checkpoint_path)
    new_checkpoint = {}

    for key in checkpoint['state_dict'].keys():
        if 'model' in key:
            new_key = key[6:]
            new_checkpoint[new_key] = checkpoint['state_dict'][key]
        else:
            new_checkpoint[key] = checkpoint['state_dict'][key]

    config = AlbertConfig.from_pretrained('albert-base-v2')
    m = AlbertForMultipleChoice.from_pretrained(pretrained_model_name_or_path= None, config=config, state_dict=new_checkpoint)
    m2 = AlbertModel.from_pretrained(pretrained_model_name_or_path= None, config=config, state_dict=new_checkpoint)

    trainer_for_test = pl.Trainer(gpus=-1, accelerator='dp')
    model_for_test = TestModel(m, test_dataloader, m2)
    
    
    i = 0
    ret_saved = 0.0
    ret_data = t_dataset
    ret_i = -1
    avg=0
    while i < runs: 
        ret = trainer_for_test.test(model=model_for_test)
        count=0
        count_c=0
        for element in t_dataset:
            if chr(element["chosen"][-1] + 65) == element["answer"]:
                count_c = count_c +1
            count = count +1
        acc = count_c/count
        avg = avg + acc
#        avg = avg + ret[0]["test_acc_epoch"]
        if acc > ret_saved:
            ret_saved = acc
            ret_data = t_dataset
            ret_i = i
        i = i+1
    avg = avg/i   
    for element in ret_data:
        element["chosen_index"] = ret_i
    pickle.dump(ret_data, open(nn_file, "wb"))  
    print("test_acc_epoch:")
    print(ret_saved)
    print("avg:")
    print(avg)

In [19]:
t_dataset = []
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2", do_lower_case=True)
test_dataloader = get_demo_dataloader(tokenizer)

07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.045s]
07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.038s]
07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.041s]
07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.041s]
07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.276s]
07/18/2021 11:16:49 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.261s]
07/18/2021 11:16:50 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.221s]
07/18/2021 11:16:50 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.221s]
07/18/2021 11:16:50 - INFO - elasticsearch -   P

Saving data to cache_demo_.cache...


In [20]:
with  open("DL_demo_.pkl", 'rb') as f:
    test_dataloader = pickle.load(f)
    
with  open("DS_demo_.pkl", 'rb') as f:
    t_dataset = pickle.load(f)
    
NN_filename = "NN-Demo-OB.pkl"
model_path = "/pfs/work7/workspace/scratch/st_st169719-TQA-0/Chen/OpenBook/dr_ex01-albert-openbook-epoch=02-val_acc_epoch=0.557.ckpt"

In [21]:
Generate_NN_file(model_path, NN_filename, 1)

Some weights of the model checkpoint at None were not used when initializing AlbertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
07/18/2021 11:16:57 - INFO - pytorch_lightning.utilities.distributed -   GPU available: True, used: True
07/18/2021 11:16:57 - INFO - pytorch_lightning.utilities.distributed -   TPU available: False, using: 0 TPU cores
07/18/2021 11:16:57 - INFO - pytorch_lightning.accelerators.gpu -   LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/pfs/data5/home/st/st_us-051200/st_st171103/env2hay/lib64/python3.6/site-pack

Testing: 0it [00:00, ?it/s]

/pfs/data5/home/st/st_us-051200/st_st171103/env2hay/lib64/python3.6/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.0, 'test_acc_epoch': 0.0}
--------------------------------------------------------------------------------
test_acc_epoch:
0.0
avg:
0.0


In [22]:
with  open("DL_demo_.pkl", 'rb') as f:
    test_dataloader = pickle.load(f)
    
with  open("DS_demo_.pkl", 'rb') as f:
    t_dataset = pickle.load(f)
    
NN_filename = "NN-Demo-Commonqa.pkl"
model_path = "/pfs/work7/workspace/scratch/st_st169719-TQA-0/Chen/Commonsense/dr_ex01-albert-commonsense-epoch=01-val_loss_epoch=1.066.ckpt"

In [23]:
Generate_NN_file(model_path, NN_filename, 1)

Some weights of the model checkpoint at None were not used when initializing AlbertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
07/18/2021 11:17:05 - INFO - pytorch_lightning.utilities.distributed -   GPU available: True, used: True
07/18/2021 11:17:05 - INFO - pytorch_lightning.utilities.distributed -   TPU available: False, using: 0 TPU cores
07/18/2021 11:17:05 - INFO - pytorch_lightning.accelerators.gpu -   LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 1.0, 'test_acc_epoch': 1.0}
--------------------------------------------------------------------------------
test_acc_epoch:
0.8
avg:
0.8


In [24]:
with  open("DL_demo_.pkl", 'rb') as f:
    test_dataloader = pickle.load(f)
    
with  open("DS_demo_.pkl", 'rb') as f:
    t_dataset = pickle.load(f)
    
NN_filename = "NN-Demo-Race.pkl"
model_path = "/pfs/work7/workspace/scratch/st_st169719-TQA-0/Chen/RACE/e3-albert-race-epoch=01-val_loss_epoch=1.16.ckpt"

In [25]:
Generate_NN_file(model_path, NN_filename, 1)

Some weights of the model checkpoint at None were not used when initializing AlbertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
07/18/2021 11:17:07 - INFO - pytorch_lightning.utilities.distributed -   GPU available: True, used: True
07/18/2021 11:17:07 - INFO - pytorch_lightning.utilities.distributed -   TPU available: False, using: 0 TPU cores
07/18/2021 11:17:07 - INFO - pytorch_lightning.accelerators.gpu -   LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.0, 'test_acc_epoch': 0.5}
--------------------------------------------------------------------------------
test_acc_epoch:
0.7
avg:
0.7


In [26]:
with  open("DL_demo_.pkl", 'rb') as f:
    test_dataloader = pickle.load(f)
    
with  open("DS_demo_.pkl", 'rb') as f:
    t_dataset = pickle.load(f)
    
NN_filename = "NN-Demo-CQA.pkl"
model_path = "/pfs/work7/workspace/scratch/st_st169719-TQA-0/Chen/Cosmos/ex02-albert-cosmos-epoch=03-val_acc_epoch=0.588.ckpt"

In [27]:
Generate_NN_file(model_path, NN_filename, 1)

Some weights of the model checkpoint at None were not used when initializing AlbertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
07/18/2021 11:17:08 - INFO - pytorch_lightning.utilities.distributed -   GPU available: True, used: True
07/18/2021 11:17:08 - INFO - pytorch_lightning.utilities.distributed -   TPU available: False, using: 0 TPU cores
07/18/2021 11:17:08 - INFO - pytorch_lightning.accelerators.gpu -   LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 1.0, 'test_acc_epoch': 0.75}
--------------------------------------------------------------------------------
test_acc_epoch:
0.8
avg:
0.8


In [28]:
import pickle
with  open("NN-Demo-CQA.pkl", 'rb') as f:
    t_dataset = pickle.load(f)

In [29]:
for element in t_dataset:
    print(element["question"])    
    print(element["options"])    
    print(element["answer"])    
    print(chr(element["chosen"][element["chosen_index"]] + 65))    
    print(element["chosen"])    
    print(element["final_optionscore"][element["chosen_index"]])    
#    print(element["final_optionscore"])    
    print(element["rscore"])        
    print(element["article"])             

Which gas do plants absorb from the atmosphere?
['oxigen', 'nitrogen', 'carbon dioxide', 'helium']
C
C
[2]
[-4.145136  -2.000894   1.9445158 -5.8803186]
[[39.45624, 39.10998, 37.332546, 35.562355], [48.692398, 41.470398, 40.56738, 39.45624], [57.20912, 50.405483, 49.27131, 49.09628], [39.45624, 39.10998, 37.332546, 35.562355]]
[' Plants are rooted in the soil, from which they absorb the minerals they need to grow, and they take up carbon dioxide gas from the atmosphere, which they convert into sugars in the process of photosynthesis.\n Plants and animals simply cannot absorb the gas directly from the atmosphere.\n For example, cyanobacteria not only restore oxygen to the atmosphere; some of them also convert nitrogen gas (N2) in the atmosphere to nitrogen compounds that plants can absorb from soil and water (see Figure 15.11 b ).\n SOME PLANTS THAT FEED ON INSECTS BY JOH\\ J. W.-\\RD, F.E.S. Plants, by means of their leaves, can absorb light and the carbonic acid gas of the atmosphere.

## CONSTANTS

In [30]:
# TRAIN_DATA_PATH =
# VAL_DATA_PATH = 
DOC_QUANTITY = 4
NUM_SCORES = 4 #rtf-idf, model1, model2, model3, model4
#LEN_DATASET = 4000
TOP_N = 20
MODELS_DIR = "model"


# BATCH_SIZE = 256
LAST_LAYER_SIZE = 768

## Getting test output data

In [31]:
### Loading test data

In [32]:
# TEST DATA
# CHALLENGE
with open('NN-Demo-CQA.pkl', 'rb') as f:
    NN_ARC_Challenge_Test_CQA4 = pickle.load(f)

with open('NN-Demo-Commonqa.pkl', 'rb') as f:
    NN_ARC_Challenge_Test_Commonqa4 = pickle.load(f)

with open('NN-Demo-OB.pkl', 'rb') as f:
    NN_ARC_Challenge_Test_OB4 = pickle.load(f)

with open('NN-Demo-Race.pkl', 'rb') as f:
    NN_ARC_Challenge_Test_Race4 = pickle.load(f)

# Prepare Transfomer Data

## Prepare input for 4 models optionscore shape (4,768)

In [33]:
def prepare_input_data_for_4_models(data_model_1, data_model_2, data_model_3, data_model_4):
    LEN_DATASET = len(data_model_1)
    data = data_model_1

    answer_a = np.zeros((LEN_DATASET, NUM_SCORES, LAST_LAYER_SIZE + 4),
                      dtype="float")
    answer_b = np.zeros((LEN_DATASET, NUM_SCORES, LAST_LAYER_SIZE + 4),
                      dtype="float")
    answer_c = np.zeros((LEN_DATASET, NUM_SCORES, LAST_LAYER_SIZE + 4),
                      dtype="float")
    answer_d = np.zeros((LEN_DATASET, NUM_SCORES, LAST_LAYER_SIZE + 4),
                      dtype="float")
    #labels = np.zeros((LEN_DATASET, DOC_QUANTITY, 4), dtype="int32")
    labels = np.zeros((LEN_DATASET, 4), dtype="int32")

  
    for i in range(0, LEN_DATASET):
    #model 1 scores 
        try:
            ms1a = data_model_1[i]['optionscore'][0] #(768,)
            ms1b = data_model_1[i]['optionscore'][1]
            ms1c = data_model_1[i]['optionscore'][2]
            ms1d = data_model_1[i]['optionscore'][3]

        except IndexError:
            # count_index_error +=1
            ms1a = 0
            ms1b = 0
            ms1c = 0
            ms1d = 0

        #model 2 scores
        try:
            ms2a = data_model_2[i]['optionscore'][0]
            ms2b = data_model_2[i]['optionscore'][1]
            ms2c = data_model_2[i]['optionscore'][2]
            ms2d = data_model_2[i]['optionscore'][3]
        except IndexError:
            # count_index_error +=1
            ms2a = 0
            ms2b = 0
            ms2c = 0
            ms2d = 0

        #model 3 scores
        try:
            ms3a = data_model_3[i]['optionscore'][0]
            ms3b = data_model_3[i]['optionscore'][1]
            ms3c = data_model_3[i]['optionscore'][2]
            ms3d = data_model_3[i]['optionscore'][3]
        except IndexError:
            # count_index_error +=1
            ms3a = 0
            ms3b = 0
            ms3c = 0
            ms3d = 0

        #model 4 scores
        try:
            ms4a = data_model_4[i]['optionscore'][0]
            ms4b = data_model_4[i]['optionscore'][1]
            ms4c = data_model_4[i]['optionscore'][2]
            ms4d = data_model_4[i]['optionscore'][3]
        except IndexError:
            # count_index_error +=1
            ms4a = 0
            ms4b = 0
            ms4c = 0
            ms4d = 0

        model_scores_a = np.array([ms1a,ms2a,ms3a,ms4a])
        model_scores_b = np.array([ms1b,ms2b,ms3b,ms4b])
        model_scores_c = np.array([ms1c,ms2c,ms3c,ms4c])
        model_scores_d = np.array([ms1d,ms2d,ms3d,ms4d])

        ###### ANSWER A ############
        # rs1 = data[i]['rscore'][0][0]
        # rs2 = data[i]['rscore'][0][1]
        # rs3 = data[i]['rscore'][0][2]
        # rs4 = data[i]['rscore'][0][3]

        # rscores = np.array([rs1,rs2,rs3,rs4])
        rscores = data[i]['rscore'][0]

        # for j in range(0, DOC_QUANTITY):
        #   i_ms = 0
        #   for k in range(0, NUM_SCORES):
        #     #qa_index, doc_index, num_score_index
        #     if k == 0:
        #       rscore_array = np.full((LAST_LAYER_SIZE), rscores[j])
        #       answer_a[i][j][k] = rscore_array
        #     else:
        #       answer_a[i][j][k] = model_scores_a[i_ms] #doc1
        #       i_ms += 1

        for k in range(0, NUM_SCORES):
          #qa_index, doc_index, num_score_index
          answer_a[i][k] = np.concatenate((model_scores_a[k], rscores), axis=None)



        ###### ANSWER B ############
        # rs1 = data[i]['rscore'][1][0]
        # rs2 = data[i]['rscore'][1][1]
        # rs3 = data[i]['rscore'][1][2]
        # rs4 = data[i]['rscore'][1][3]

        # rscores = np.array([rs1,rs2,rs3,rs4])

        rscores = data[i]['rscore'][1]


        # for j in range(0, DOC_QUANTITY):
        #   i_ms = 0
        #   for k in range(0, NUM_SCORES):
        #     #qa_index, doc_index, num_score_index
        #     if k == 0:
        #       rscore_array = np.full((LAST_LAYER_SIZE), rscores[j])
        #       answer_b[i][j][k] = rscore_array
        #     else:
        #       answer_b[i][j][k] = model_scores_b[i_ms] #doc1
        #       i_ms += 1

        for k in range(0, NUM_SCORES):
          #qa_index, doc_index, num_score_index
          answer_b[i][k] = np.concatenate((model_scores_b[k], rscores), axis=None)



        ###### ANSWER C ############
        # rs1 = data[i]['rscore'][2][0]
        # rs2 = data[i]['rscore'][2][1]
        # rs3 = data[i]['rscore'][2][2]
        # rs4 = data[i]['rscore'][2][3]

        # rscores = np.array([rs1,rs2,rs3,rs4])

        rscores = data[i]['rscore'][2]


        # for j in range(0, DOC_QUANTITY):
        #   i_ms = 0
        #   for k in range(0, NUM_SCORES):
        #     #qa_index, doc_index, num_score_index
        #     if k == 0:
        #       rscore_array = np.full((LAST_LAYER_SIZE), rscores[j])
        #       answer_c[i][j][k] = rscore_array
        #     else:
        #       answer_c[i][j][k] = model_scores_c[i_ms] #doc1
        #       i_ms += 1

        for k in range(0, NUM_SCORES):
          #qa_index, doc_index, num_score_index
          answer_c[i][k] = np.concatenate((model_scores_c[k], rscores), axis=None)


        ###### ANSWER D ############
        # rs1 = data[i]['rscore'][3][0]
        # rs2 = data[i]['rscore'][3][1]
        # rs3 = data[i]['rscore'][3][2]
        # rs4 = data[i]['rscore'][3][3]

        # rscores = np.array([rs1,rs2,rs3,rs4])

        rscores = data[i]['rscore'][3]


        # for j in range(0, DOC_QUANTITY):
        #   i_ms = 0
        #   for k in range(0, NUM_SCORES):
        #     #qa_index, doc_index, num_score_index
        #     if k == 0:
        #       rscore_array = np.full((LAST_LAYER_SIZE), rscores[j])
        #       answer_d[i][j][k] = rscore_array
        #     else:
        #       answer_d[i][j][k] = model_scores_d[i_ms] #doc1
        #       i_ms += 1


        for k in range(0, NUM_SCORES):
          # qa_index, doc_index, num_score_index
          answer_d[i][k] = np.concatenate((model_scores_d[k], rscores), axis=None)


        ###### CORRECT ANSWER ############

        answer = data[i]['answer']
        if answer == 'a' or answer == 'A' or answer == '1' or answer == 1:
            a_index = 0
        elif answer == 'b' or answer == 'B' or answer == '2' or answer == 2:
            a_index = 1
        elif answer == 'c' or answer == 'C' or answer == '3' or answer == 3:
            a_index = 2
        elif answer == 'd' or answer == 'D' or answer == '4' or answer == 4:
            a_index = 3
        else:
            a_index = 5 #or random number between 0 and 3 for a,b,c,d

        # labels[i][0][a_index] = 1
        # labels[i][1][a_index] = 1
        # labels[i][2][a_index] = 1
        # labels[i][3][a_index] = 1
        labels[i][a_index] = 1

    return {
          "answer_a": answer_a,
          "answer_b": answer_b,
          "answer_c": answer_c,
          "answer_d": answer_d
    }, labels

# train_data_input_easy = prepare_input_data_for_4_models(NN_ARC_Easy_Train_CQA4, NN_ARC_Easy_Train_Commonqa4, NN_ARC_Easy_Train_OB4, NN_ARC_Easy_Train_Race4)
# dev_data_input_easy = prepare_input_data_for_4_models(NN_ARC_Easy_Dev_CQA4, NN_ARC_Easy_Dev_Commonqa4, NN_ARC_Easy_Dev_OB4, NN_ARC_Easy_Dev_Race4)
#test_data_input_easy = prepare_input_data_for_4_models(NN_ARC_Easy_Test_CQA4, NN_ARC_Easy_Test_Commonqa4, NN_ARC_Easy_Test_OB4, NN_ARC_Easy_Test_Race4)

# train_data_input_challenge = prepare_input_data_for_4_models(NN_ARC_Challenge_Train_CQA4, NN_ARC_Challenge_Train_Commonqa4, NN_ARC_Challenge_Train_OB4, NN_ARC_Challenge_Train_Race4)
# dev_data_input_challenge = prepare_input_data_for_4_models(NN_ARC_Challenge_Dev_CQA4, NN_ARC_Challenge_Dev_Commonqa4, NN_ARC_Challenge_Dev_OB4, NN_ARC_Challenge_Dev_Race4)
test_data_input = prepare_input_data_for_4_models(NN_ARC_Challenge_Test_CQA4, NN_ARC_Challenge_Test_Commonqa4, NN_ARC_Challenge_Test_OB4, NN_ARC_Challenge_Test_Race4)

def concatenate_two_sets(set1, set2):
    answer_a = np.concatenate((set1[0]['answer_a'],set2[0]['answer_a']))
    answer_b = np.concatenate((set1[0]['answer_b'],set2[0]['answer_b']))
    answer_c = np.concatenate((set1[0]['answer_c'],set2[0]['answer_c']))
    answer_d = np.concatenate((set1[0]['answer_d'],set2[0]['answer_d']))
    labels = np.concatenate((set1[1], set2[1]))

    return {'answer_a':answer_a,
      'answer_b':answer_b,
      'answer_c':answer_c,
      'answer_d':answer_d
    }, labels


#train_data_input = concatenate_two_sets(train_data_input_easy, train_data_input_challenge)
#dev_data_input = concatenate_two_sets(dev_data_input_easy, dev_data_input_challenge)
#test_data_input = concatenate_two_sets(test_data_input_easy, test_data_input_challenge)

#print(test_data_input[0])
# print(train_data_input_easy[0])

# Neural network setup

## MODEL

### experiment arch for reshaped inputs

In [34]:
# from arch.py

model_quantitiy = 4

def get_model():

    if model_quantitiy == 3:
        print("model_quantitiy: " + str(model_quantitiy))
        answer_a = Input(shape=(DOC_QUANTITY , NUM_SCORES + 1), name="answer_a")
        answer_b = Input(shape=(DOC_QUANTITY , NUM_SCORES + 1), name="answer_b") 
        answer_c = Input(shape=(DOC_QUANTITY , NUM_SCORES + 1), name="answer_c") 
        answer_d = Input(shape=(DOC_QUANTITY , NUM_SCORES + 1), name="answer_d") 

    elif model_quantitiy == 4:
        print("model_quantitiy: " + str(model_quantitiy))
        answer_a = Input(shape=(NUM_SCORES , LAST_LAYER_SIZE + 4), name="answer_a") #4,5*768
        answer_b = Input(shape=(NUM_SCORES , LAST_LAYER_SIZE + 4), name="answer_b") 
        answer_c = Input(shape=(NUM_SCORES , LAST_LAYER_SIZE + 4), name="answer_c") 
        answer_d = Input(shape=(NUM_SCORES , LAST_LAYER_SIZE + 4), name="answer_d")    
    else:
        print("model_quantitiy: " + str(model_quantitiy))
        answer_a = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_a") #4,3
        answer_b = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_b") 
        answer_c = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_c") 
        answer_d = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_d") 
    # These layers are shared for each answer.
    # apply a layer to every temporal slice of an input
    #  The input should be at least 3D, and the dimension of index one
    #  will be considered to be the temporal dimension.
    encoder_layer1 = TimeDistributed(
                      Dense(32, activation='tanh', name="dense_1"),
                      name="time_distributed_1"
    )
    # if model_quantitiy == 4:
    #   dropout_layer = SpatialDropout2D(0.25, name="spatial_dropout_1") 
    # else:
    #   dropout_layer = SpatialDropout1D(0.25, name="spatial_dropout_1")
    dropout_layer = SpatialDropout1D(0.15, name="spatial_dropout_1")

    # implementing lstm
    # input_dim = 3
    # hidden_dim = 32
    # n_layers = 100

    #lstm_layer = LSTM(return_sequences=False, dropout = 0.225, units = 4) #do it in keras!!!
    bilstm_layer = Bidirectional(LSTM(return_sequences=False, dropout = 0.275, units = 4)) #do it in keras!!! input_shape=(5,768)
    # gru_layer = Bidirectional(GRU(return_sequences=False, dropout = 0.25, units = 4))
    # convLSTM2d_layer = ConvLSTM2D(filters = 4, kernel_size = (5,768), return_sequences=False, dropout = 0.225)


    # to_scalar_layer = KVAttention(
    #                     key_size=64, value_size=8, name="att",
    #                     return_attention_scores=return_attention_scores
    # )
    def encode_answer(answer):
        x = encoder_layer1(answer)
        x = dropout_layer(x)
        #x = lstm_layer(x)
        x = bilstm_layer(x)
        # x = gru_layer(x)
        #x = convLSTM2d_layer(x)
        #x = to_scalar_layer(x)
        return x
    a = encode_answer(answer_a)
    b = encode_answer(answer_b)
    c = encode_answer(answer_c)
    d = encode_answer(answer_d)
    y = Concatenate(axis=-1, name="concatenate_answers")([a, b, c, d])
    # if return_attention_scores:
    #   output = y
    # else:


    y = Dense(16, activation='relu', name="dense_2")(y)
    y = Dropout(0.1, name="dropout_1")(y)
    y = Dense(16, activation='relu', name="dense_3")(y)
    y = Dropout(0.1, name="dropout_2")(y)
    y = Dense(16, activation='relu', name="dense_4")(y)

    output = Dense(4, activation='softmax', name="dense_5")(y)

    model = Model(inputs=[answer_a, answer_b, answer_c, answer_d],
                    outputs=[output])

    model.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                # optimizer = sgd,
                # metrics = ['accuracy'])
                metrics = ['acc'])
    # new_model.summary()
    # plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    return model

## Evaluate trained Model on Test Set

### fit model

In [35]:
# eval.py
def fit(file_name):
    model_weights = None
    model_weights = os.path.join('', str(file_name))  # noqa: E502
    assert(model_weights is not None)

    # easy and dev combined
    data, labels = test_data_input
    # LEN_DATASET_VAL = len(data['answer_a'])
    # re_val_answer_a = data['answer_a'].reshape(LEN_DATASET_VAL, NUM_SCORES, LAST_LAYER_SIZE)
    # re_val_answer_b = data['answer_b'].reshape(LEN_DATASET_VAL, NUM_SCORES, LAST_LAYER_SIZE)
    # re_val_answer_c = data['answer_c'].reshape(LEN_DATASET_VAL, NUM_SCORES, LAST_LAYER_SIZE)
    # re_val_answer_d = data['answer_d'].reshape(LEN_DATASET_VAL, NUM_SCORES, LAST_LAYER_SIZE)
    # re_val_data = {'answer_a':re_val_answer_a,
    #         'answer_b':re_val_answer_b,
    #         'answer_c':re_val_answer_c,
    #         'answer_d':re_val_answer_d
    #       }



    model = get_model()
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    model.load_weights(model_weights)
    #model.summary()
    y = model.predict(data, batch_size=24)


    num_questions = labels.shape[0]
    correct = 0
    for i in range(0, num_questions):
        expected = np.argmax(labels[i])
        predicted = np.argmax(y[i])
        if expected == predicted:
            correct += 1

    accuracy = 100.0 * (correct / num_questions)
    print("Accuracy: {0:.4f}%".format(100.0 * correct / num_questions))

    return y,accuracy

nn_predicted = fit("content/model/20210627-175001/model-0372-1.1408-0.5273.hdf5")

# if __name__ == "__main__":
#     main()

model_quantitiy: 4
Accuracy: 80.0000%


In [36]:
# CHALLENGE
len_set = len(NN_ARC_Challenge_Test_OB4)
count_at_least_one_model_answered_correctly_challenge = 0
correct_cqa_challenge = 0
correct_commonqa_challenge = 0
correct_ob_challenge = 0
correct_race_challenge = 0
check_arr = []

for qa_index in range(0, len_set):
    chosen_array_index_cqa = NN_ARC_Challenge_Test_CQA4[qa_index]['chosen_index']
    answer_of_tm_cqa = NN_ARC_Challenge_Test_CQA4[qa_index]['chosen'][chosen_array_index_cqa]

    chosen_array_index_commonqa = NN_ARC_Challenge_Test_Commonqa4[qa_index]['chosen_index']
    answer_of_tm_commonqa = NN_ARC_Challenge_Test_Commonqa4[qa_index]['chosen'][chosen_array_index_commonqa]

    chosen_array_index_ob = NN_ARC_Challenge_Test_OB4[qa_index]['chosen_index']
    answer_of_tm_ob4 = NN_ARC_Challenge_Test_OB4[qa_index]['chosen'][chosen_array_index_ob]

    chosen_array_index_race = NN_ARC_Challenge_Test_Race4[qa_index]['chosen_index']
    answer_of_tm_race = NN_ARC_Challenge_Test_Race4[qa_index]['chosen'][chosen_array_index_race]
    
    answer = NN_ARC_Challenge_Test_OB4[qa_index]['answer']
    if answer == 'a' or answer == 'A' or answer == '1' or answer == 1:
        a_index = 0
    elif answer == 'b' or answer == 'B' or answer == '2' or answer == 2:
        a_index = 1
    elif answer == 'c' or answer == 'C' or answer == '3' or answer == 3:
        a_index = 2
    elif answer == 'd' or answer == 'D' or answer == '4' or answer == 4:
        a_index = 3
    else:
        a_index = 5 #or random number between 0 and 3 for a,b,c,d
    
    check_arr.append([NN_ARC_Challenge_Test_OB4[qa_index]['question'],NN_ARC_Challenge_Test_OB4[qa_index]['options'],answer,[answer_of_tm_cqa,answer_of_tm_commonqa,answer_of_tm_race,answer_of_tm_ob4]])
    
    if answer_of_tm_cqa == a_index:
        correct_cqa_challenge +=1

    if answer_of_tm_commonqa == a_index:
        correct_commonqa_challenge += 1

    if answer_of_tm_ob4 == a_index:
        correct_ob_challenge += 1

    if answer_of_tm_race == a_index:
        correct_race_challenge += 1

    if answer_of_tm_cqa == a_index:
        count_at_least_one_model_answered_correctly_challenge += 1
        continue
    
    if answer_of_tm_commonqa == a_index:
        count_at_least_one_model_answered_correctly_challenge += 1
        continue
    
    if answer_of_tm_ob4 == a_index:
        count_at_least_one_model_answered_correctly_challenge += 1
        continue
    
    if answer_of_tm_race == a_index:
        count_at_least_one_model_answered_correctly_challenge += 1
        continue


at_least_one_correct_answer_challenge_acc = count_at_least_one_model_answered_correctly_challenge/len_set
correct_cqa_challenge_acc = correct_cqa_challenge/len_set
correct_commonqa_challenge_acc = correct_commonqa_challenge/len_set
correct_ob_challenge_acc = correct_ob_challenge/len_set
correct_race_challenge_acc = correct_race_challenge/len_set


print('at least one: ' + str(at_least_one_correct_answer_challenge_acc))
print('cqa: ' + str(correct_cqa_challenge_acc))
print('commonqa: ' + str(correct_commonqa_challenge_acc))
print('ob: ' + str(correct_ob_challenge_acc))
print('race: ' + str(correct_race_challenge_acc))

at least one: 1.0
cqa: 0.8
commonqa: 0.8
ob: 0.0
race: 0.7


In [37]:
print("CQA|ComQA|Race|OB")
print(" ")
for element in check_arr:
    for sub in element:
        print(sub)           

CQA|ComQA|Race|OB
 
Which gas do plants absorb from the atmosphere?
['oxigen', 'nitrogen', 'carbon dioxide', 'helium']
C
[2, 2, 2, 0]
What colour do you get if you mix blue and yellow paint together?
['red', 'green', 'blue', 'yellow']
B
[2, 2, 1, 3]
What is the sweet substance made by bees?
['beeswax', 'pollen', 'propolis', 'honey']
D
[3, 3, 3, 2]
What is the only metal that is liquid at room temperature?
['mercury', 'copper', 'gold', 'silver']
A
[0, 0, 0, 1]
What is the name of the largest ocean on Earth?
['The Pacific Ocean', 'The Indian Ocean', 'The Atlantic Ocean', 'The Arctic Ocean']
A
[2, 0, 1, 2]
What do you call a person who studies rocks?
['mineralogist', 'geologist', 'vulcanologist', 'speologist']
B
[1, 1, 3, 0]
What country does parmesan cheese come from?
['Germany', 'France', 'Italy', 'Switzerland']
C
[2, 2, 2, 1]
A thermometer is a device used to measure what?
['length', 'height', 'distance', 'temperature']
D
[3, 3, 3, 0]
What is the name of the part of the human skeleton 